### Introduction with Spanner

Cloud Spanner is a fully managed, mission-critical, relational database service that offers transactional consistency at global scale, schemas, SQL (ANSI 2011 with extensions), and automatic, synchronous replication for high availability. 

When we use cloud to process relational data like `Mysql` to give each row with index, we would like to use `Spanner`. To compare `Spanner` with `Mysql` and `NOSQL`, you could get the unstanding with this image:
![spanner compare](https://docs.google.com/uc?export=download&id=1wsq3aD2jNwBMwPU0rD0DgPeGgBnpBAWV)

This tutorial is mainly used to show how we could use python to interact with `Spanner`, let's start.

In [2]:
# first let's install spanner API
! pip install google-cloud-spanner==1.17.0 --quiet

     |████████████████████████████████| 256kB 4.6MB/s 


In [3]:
# first let's set project
! gcloud config set project cloudtutorial-279003

Updated property [core/project].


In [4]:
# let's auth this notebook
from google.colab import auth
auth.authenticate_user()

In [5]:
# first let's try to create a database first
# first let's init the spanner client
project_id = "cloudtutorial-279003"

from google.cloud import spanner

client = spanner.Client(project_id)

### Create database

In [6]:
# before we create database, first we should create instance.
# config is to set the region with how many nodes =1
! gcloud spanner instances create first-instance --config=regional-us-central1 --description='first instance' --nodes=1

In [7]:
# delete database first, if we face doesn't exist error, doesn't matter
! gcloud spanner databases delete first-db --instance=first-instance 

You are about to delete database: [first-db]

Do you want to continue (Y/n)?  y

ERROR: (gcloud.spanner.databases.delete) NOT_FOUND: Database not found: projects/cloudtutorial-279003/instances/first-instance/databases/first-db
- '@type': type.googleapis.com/google.rpc.ResourceInfo
  description: Database does not exist.
  resourceName: projects/cloudtutorial-279003/instances/first-instance/databases/first-db
  resourceType: type.googleapis.com/google.spanner.admin.database.v1.Database


In [11]:
# alright, let's try to create a database 
# first we should provide instance_id and database_id
instance_id = "first-instance"
database_id = "first-db"


# init instance
instance = client.instance(instance_id=instance_id)

In [12]:
# delete table in case exists with instance obejct.
database = instance.database(database_id, ddl_statements=["delete table users if exists"])

In [13]:
# then with database object is based on instance, ddl_statements is create sql
database = instance.database(database_id, ddl_statements=[
                                                      """create table users(
                                                        user_id int64 not null,
                                                        first_name string(128), 
                                                        last_name string(128),
                                                        other_info string(128)
                                                      ) primary key (user_id)
                                                      """    
])

# let's create database
operation = database.create()

print("waiting create database to finish")
operation.result(120)

print("instance: {} created database:{}".format(instance_id, database_id))

waiting create database to finish
instance: first-instance created database:first-db


In [14]:
# let's first check with the created database
# we do get the result data.
with database.snapshot() as snapshot:
  res = snapshot.execute_sql("select 1")
  for r in res:
    print(r)

[1]


### Insert values into tables

In [15]:
# next step let's do the insert function
insert_sql = "insert users(user_id, first_name, last_name, other_info) values {}"

value_list = []
for i in range(5):
  value_list.append("({}, '{}', '{}', '{}'), ".format(i, "lu_" + str(i), "gq_" + str(i), str(i+10)))

value_str = "".join(value_list)
value_str = value_str[:-2] 
insert_sql = insert_sql.format(value_str)

print("insert sql:", insert_sql)

insert sql: insert users(user_id, first_name, last_name, other_info) values (0, 'lu_0', 'gq_0', '10'), (1, 'lu_1', 'gq_1', '11'), (2, 'lu_2', 'gq_2', '12'), (3, 'lu_3', 'gq_3', '13'), (4, 'lu_4', 'gq_4', '14')


In [16]:
# let's create a function to do insert
def insert_user(transaction):
  row = transaction.execute_update(insert_sql)
  print("{} records inserted.".format(row))

# run_in_transaction: Perform a unit of work in a transaction, retrying on abort.
database.run_in_transaction(insert_user)

5 records inserted.


In [17]:
# in fact, we could also use batch insert logic to insert many rows once, that's batch insert
values_list = []
for i in range(6, 10):
  values_list.append((i, "liu_" + str(i), "ye" + str(i), 'no other'))

with database.batch() as batch:
  batch.insert(table='users', columns=['user_id', 'first_name', 'last_name', 'other_info'], values=values_list)

print("insert finished.")

insert finished.


### Query data from table

In [18]:
# we could use command line to execute query in fact
! gcloud spanner databases execute-sql first-db --instance=first-instance --sql="select * from users"

user_id  first_name  last_name  other_info
0        lu_0        gq_0       10
1        lu_1        gq_1       11
2        lu_2        gq_2       12
3        lu_3        gq_3       13
4        lu_4        gq_4       14
6        liu_6       ye6        no other
7        liu_7       ye7        no other
8        liu_8       ye8        no other
9        liu_9       ye9        no other


In [19]:
# let's get result with client
query_sql = "select * from users"

with database.snapshot() as snapshot:
  res = snapshot.execute_sql(query_sql)

  for row in res:
    print("user_id:{}, first_name: {}, last_name: {}, other_info: {}".format(*row))

user_id:0, first_name: lu_0, last_name: gq_0, other_info: 10
user_id:1, first_name: lu_1, last_name: gq_1, other_info: 11
user_id:2, first_name: lu_2, last_name: gq_2, other_info: 12
user_id:3, first_name: lu_3, last_name: gq_3, other_info: 13
user_id:4, first_name: lu_4, last_name: gq_4, other_info: 14
user_id:6, first_name: liu_6, last_name: ye6, other_info: no other
user_id:7, first_name: liu_7, last_name: ye7, other_info: no other
user_id:8, first_name: liu_8, last_name: ye8, other_info: no other
user_id:9, first_name: liu_9, last_name: ye9, other_info: no other


In [20]:
# in fact, we could also do query with filter in sql, let's do it
with database.snapshot() as snapshot:
  res = snapshot.execute_sql("select * from users where first_name= @first_name", 
                             params={"first_name": "lu_0"}, 
                             param_types={"first_name":  spanner.param_types.STRING})
  # let's check the result
  # so we do get the result from spanner with filter.
  for row in res:
    print("user_id: {}, first_name: {}, last_name:{}, other_info: {}".format(*row))

user_id: 0, first_name: lu_0, last_name:gq_0, other_info: 10


In [21]:
# above steps are both use SQL to get data, in fact we could also use API to read data
with database.snapshot() as snapshot:
  keyset = spanner.KeySet(all_=True)
  res = snapshot.read(table='users', columns=('user_id', "first_name"), keyset=keyset)

  for row in res:
    print("user_id: {}, first_name:{}".format(*row))

user_id: 0, first_name:lu_0
user_id: 1, first_name:lu_1
user_id: 2, first_name:lu_2
user_id: 3, first_name:lu_3
user_id: 4, first_name:lu_4
user_id: 6, first_name:liu_6
user_id: 7, first_name:liu_7
user_id: 8, first_name:liu_8
user_id: 9, first_name:liu_9


### Update table with some rows

In [22]:
# update the table with some rows like MySQL
with database.batch() as batch:
  batch.update(table="users", columns=("user_id", "first_name", "last_name", "other_info"), values=[(0, "lu", "gq", 'nothing')])

# let's check
with database.snapshot() as snapshot:
  res = snapshot.execute_sql("select * from users where user_id= @user_id", 
                             params={"user_id": 0}, 
                             param_types={"user_id": spanner.param_types.INT64})

  # so we do update the user_id = 0 value....
  for row in res:
    print("user_id: {}, first_name: {}, last_name: {}, other: {}".format(*row))

user_id: 0, first_name: lu, last_name: gq, other: nothing


### Add second index

When we say `index` in database, we would like to infer that we could use `index` to do query without scanning whole tables just like `MySQL`.

In [23]:
# let's add second index on the first_name column
# command: create index `index_name` on `table`(`column`)
update_index_sql = "create index first_name_index on users(first_name)"

# update_ddl should provide with  a list of SQL
operation = database.update_ddl([update_index_sql])

print("waiting operation to finish")
operation.result(120)

print("Already add index")

waiting operation to finish
Already add index


In [24]:
# let's read data with index
with database.snapshot() as snapshot:
  keyset = spanner.KeySet(all_=True)
  # one thing to notice: when we use API to read data, if we need to query not index column will raise error.
  res = snapshot.read(table='users', 
                      columns=('user_id', 'first_name'), 
                      keyset=keyset, 
                      index='first_name_index')

  for row in res:
    print("user_id:  {}, first_name: {}".format(*row))

user_id:  6, first_name: liu_6
user_id:  7, first_name: liu_7
user_id:  8, first_name: liu_8
user_id:  9, first_name: liu_9
user_id:  0, first_name: lu
user_id:  1, first_name: lu_1
user_id:  2, first_name: lu_2
user_id:  3, first_name: lu_3
user_id:  4, first_name: lu_4


#### One thing to notice to create index

as you could see that if we add index on `first_column`, we could't select other columns, if we do need to select other columns, we should create storing columns, we have to create a `new index name`, don't allow second time index with same index name.

In [25]:
new_update_index_sql = "create index first_name_index_new on users(first_name) storing (last_name, other_info)"

operation = database.update_ddl([new_update_index_sql])

print("waiting to finish")
operation.result(120)

print("new index created")

waiting to finish
new index created


In [26]:
# let's check with new index
with database.snapshot() as snapshot:
  keyset = spanner.KeySet(all_=True)
  # so that we could query some columns we want.
  res = snapshot.read(table='users', 
                      columns=('user_id', 'first_name', 'last_name', 'other_info'), 
                      keyset=keyset, 
                      index='first_name_index_new')

  for row in res:
    print("user_id:  {}, first_name: {}, last_name: {}, other_info:{}".format(*row))


user_id:  6, first_name: liu_6, last_name: ye6, other_info:no other
user_id:  7, first_name: liu_7, last_name: ye7, other_info:no other
user_id:  8, first_name: liu_8, last_name: ye8, other_info:no other
user_id:  9, first_name: liu_9, last_name: ye9, other_info:no other
user_id:  0, first_name: lu, last_name: gq, other_info:nothing
user_id:  1, first_name: lu_1, last_name: gq_1, other_info:11
user_id:  2, first_name: lu_2, last_name: gq_2, other_info:12
user_id:  3, first_name: lu_3, last_name: gq_3, other_info:13
user_id:  4, first_name: lu_4, last_name: gq_4, other_info:14


### Read-only transaction

In multi-thread application, sometimes, we need to ensure every transaction should get same result with different thread, so `spanner` also support `read-only transaction`.

In [27]:
# we could use sql or api to read data and to ensure we get same result for each read
# we have to set `multi_use=True`, so that we could use snapshot to do query again.
with database.snapshot(multi_use=True) as snapshot:
  # read with SQL
  res = snapshot.execute_sql("select * from users where user_id=0")
  print("First read:")
  for row in res:
    print("user_id:  {}, first_name: {}, last_name: {}, other_info:{}".format(*row))
  
  # let's try to update the user_id with 0 to check with same session to read data should get same result
  with database.batch() as batch:
    batch.update(table="users", columns=("user_id", "first_name", "last_name", "other_info"), values=[(0, "lu_change", "gq", 'nothing')])
  
  # read data with API
  keyset = spanner.KeySet(all_=True)
  res = snapshot.read(table='users', columns=('user_id', 'first_name', 'last_name', 'other_info'), keyset=keyset, )

  print("second read:")
  for row in res:
    print("user_id:  {}, first_name: {}, last_name: {}, other_info:{}".format(*row))

First read:
user_id:  0, first_name: lu, last_name: gq, other_info:nothing
second read:
user_id:  0, first_name: lu, last_name: gq, other_info:nothing
user_id:  1, first_name: lu_1, last_name: gq_1, other_info:11
user_id:  2, first_name: lu_2, last_name: gq_2, other_info:12
user_id:  3, first_name: lu_3, last_name: gq_3, other_info:13
user_id:  4, first_name: lu_4, last_name: gq_4, other_info:14
user_id:  6, first_name: liu_6, last_name: ye6, other_info:no other
user_id:  7, first_name: liu_7, last_name: ye7, other_info:no other
user_id:  8, first_name: liu_8, last_name: ye8, other_info:no other
user_id:  9, first_name: liu_9, last_name: ye9, other_info:no other


So we do get same result in one snapshot for changed record even we have changed the data in the same thread.

### Delete records


In [29]:
# let's create a function to do the query with spanner
def query_table(table_name='users'):
  with database.snapshot(multi_use=True) as snapshot:
  # read with SQL
    res = snapshot.execute_sql("select * from {}".format(table_name))
    print("Finish reading:")
    for row in res:
      print("user_id:  {}, first_name: {}, last_name: {}, other_info:{}".format(*row))

In [37]:
# we could delete one records
one_row_to_delete = spanner.KeySet(keys=[[2]])

with database.batch() as batch:
  batch.delete("users", one_row_to_delete)


#### KeySet

If you are curious about KeySet, you could find it [here](https://cloud.google.com/spanner/docs/reference/rest/v1/KeySet).

In [38]:
  # let's check,  so that we do delete the row_id with 2
  query_table()

Finish reading:
user_id:  3, first_name: lu_3, last_name: gq_3, other_info:13
user_id:  4, first_name: lu_4, last_name: gq_4, other_info:14
user_id:  6, first_name: liu_6, last_name: ye6, other_info:no other
user_id:  7, first_name: liu_7, last_name: ye7, other_info:no other
user_id:  8, first_name: liu_8, last_name: ye8, other_info:no other
user_id:  9, first_name: liu_9, last_name: ye9, other_info:no other


In [42]:
# in fact, we could also delete a range of rows
# start_closed means include; end_open means not include
many_rows_to_delete = spanner.KeyRange(start_closed=[3], end_open=[5])
# then we have to convert into keyset
many_rows_keyset = spanner.KeySet(ranges=[many_rows_to_delete])

with database.batch() as batch:
  batch.delete("users",  many_rows_keyset)

In [43]:
  # let's check,  so that we do delete the row_id range from 3 to 5
  query_table()

Finish reading:
user_id:  6, first_name: liu_6, last_name: ye6, other_info:no other
user_id:  7, first_name: liu_7, last_name: ye7, other_info:no other
user_id:  8, first_name: liu_8, last_name: ye8, other_info:no other
user_id:  9, first_name: liu_9, last_name: ye9, other_info:no other


In [44]:
#last we could delete the instance we created.
! gcloud spanner instances delete first-instance

Delete instance [first-instance]. Are you sure?

Do you want to continue (Y/n)?  y

